<a href="https://colab.research.google.com/github/denisbebrovich/ML-DS/blob/main/DefLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
  data = []
  labels = []
  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size
  print(start_index, end_index, target.shape)
  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

def lstm_pred(train, test):
  train_array = train.values
  train_array = train_array[:, [0, 1, 2]]
  test_array = test.values
  test_array = test_array[:, [0, 1, 2]]
  history_size = 720
  target_size = 1
  step = 6
  TRAIN_SPLIT = len(train_array) * 0.8
  tf.random.set_seed(13)
  EVALUATION_INTERVAL = 200
  EPOCHS = 10

  x_train_single, y_train_single = multivariate_data(train_array, train_array, 0,
                                                   TRAIN_SPLIT, history_size,
                                                   target_size, step,
                                                   single_step=True)

  x_val_single, y_val_single = multivariate_data(train_array, train_array,
                                                   TRAIN_SPLIT, None, history_size,
                                                   target_size, step,
                                                   single_step=True)

  x_test_single, y_test_single = multivariate_data(test_array, test_array,
                                               0, None, history_size,
                                               target_size, step,
                                               single_step=True)

  BATCH_SIZE = 256
  BUFFER_SIZE = 5000

  train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
  train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

  test_data_single = tf.data.Dataset.from_tensor_slices((x_test_single, y_test_single))
  test_data_single = test_data_single.batch(BATCH_SIZE).repeat()

  val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
  val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

  single_step_model = tf.keras.models.Sequential()
  single_step_model.add(tf.keras.layers.LSTM(32, input_shape=x_train_single.shape[-2:]))
  single_step_model.add(tf.keras.layers.Dense(3))
  single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

  single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)
  predict = single_step_model.predict(x_train_single)
  train_predict = single_step_model.predict(x_train_single)

  train_abs_Dif = np.array(abs(train_predict - y_train_single))
  train_abs_Dif_sum = train_abs_Dif.sum(axis = 1)
  treshold = np.amax(train_abs_Dif_sum, axis = None)
  test_predict = single_step_model.predict(x_test_single)
  test_abs_Dif_sum = test_abs_Dif.sum(axis = 1)

  returned_arr = np.array(list(map(lambda x: int(x > treshold), test_abs_Dif_sum)))
  returned_arr[returned_arr == 1] = -1
  returned_arr[returned_arr == 0] = 1

  zeros = np.zeros((720,), dtype=int)
  zeros[zeros == 0] = 1
  returner = np.concatenate((zeros, returned_arr), axis=None)
  return returner